In [1]:
import pandas as pd
import io
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
print ('Ran')

Ran


In [2]:
prev_app = pd.read_csv('D:/Project_Data/previous_application.csv')

In [3]:
prev_app.shape

(1670214, 37)

In [4]:
prev_app=prev_app.iloc[0:1048575,:]

In [5]:
prev_app.shape

(1048575, 37)

In [6]:
flag_col=[col for col in prev_app.columns if 'FLAG_' in col]

In [7]:
flag_col

['FLAG_LAST_APPL_PER_CONTRACT',
 'NFLAG_LAST_APPL_IN_DAY',
 'NFLAG_INSURED_ON_APPROVAL']

In [8]:
cat_features = [f for f in prev_app.columns if prev_app[f].dtype == 'object' and f not in flag_col]

In [9]:
cat_features

['NAME_CONTRACT_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'NAME_CASH_LOAN_PURPOSE',
 'NAME_CONTRACT_STATUS',
 'NAME_PAYMENT_TYPE',
 'CODE_REJECT_REASON',
 'NAME_TYPE_SUITE',
 'NAME_CLIENT_TYPE',
 'NAME_GOODS_CATEGORY',
 'NAME_PORTFOLIO',
 'NAME_PRODUCT_TYPE',
 'CHANNEL_TYPE',
 'NAME_SELLER_INDUSTRY',
 'NAME_YIELD_GROUP',
 'PRODUCT_COMBINATION']

In [10]:
num_features = [f for f in prev_app.columns if prev_app[f].dtype != 'object' and f not in flag_col]

In [11]:
num_features

['SK_ID_PREV',
 'SK_ID_CURR',
 'AMT_ANNUITY',
 'AMT_APPLICATION',
 'AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
 'AMT_GOODS_PRICE',
 'HOUR_APPR_PROCESS_START',
 'RATE_DOWN_PAYMENT',
 'RATE_INTEREST_PRIMARY',
 'RATE_INTEREST_PRIVILEGED',
 'DAYS_DECISION',
 'SELLERPLACE_AREA',
 'CNT_PAYMENT',
 'DAYS_FIRST_DRAWING',
 'DAYS_FIRST_DUE',
 'DAYS_LAST_DUE_1ST_VERSION',
 'DAYS_LAST_DUE',
 'DAYS_TERMINATION']

In [12]:
df = prev_app[num_features]

In [13]:
df.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,0.0,0.182832,0.867336,-73,35,12.0,365243.0,-42.0,300.0,-42.0,-37.0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,NaN,NaN,NaN,-164,-1,36.0,365243.0,-134.0,916.0,365243.0,365243.0
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,11,NaN,NaN,NaN,-301,-1,12.0,365243.0,-271.0,59.0,365243.0,365243.0
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,7,NaN,NaN,NaN,-512,-1,12.0,365243.0,-482.0,-152.0,-182.0,-177.0
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,9,NaN,NaN,NaN,-781,-1,24.0,NaN,NaN,NaN,NaN,NaN


In [14]:
def agg_numeric(df, group_var):

    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = df.groupby(group_var).agg('mean').reset_index()

    return agg

In [15]:
num_group_df = agg_numeric(df.drop(columns = ['SK_ID_PREV']), group_var = 'SK_ID_CURR')

In [16]:
num_group_df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION
0,100001,3951.000,24835.50,23787.00,2520.0,24835.5,13.0,0.104326,NaN,NaN,-1740.0,23.0,8.0,365243.0,-1709.000000,-1499.000000,-1619.0,-1612.000000
1,100002,9251.775,179055.00,179055.00,0.0,179055.0,9.0,0.000000,NaN,NaN,-606.0,500.0,24.0,365243.0,-565.000000,125.000000,-25.0,-17.000000
2,100003,81462.330,618750.00,692259.75,0.0,618750.0,14.5,0.000000,NaN,NaN,-787.0,699.5,9.0,365243.0,-756.500000,-516.500000,-591.5,-583.000000
3,100004,5357.250,24282.00,20106.00,4860.0,24282.0,5.0,0.212008,NaN,NaN,-815.0,30.0,4.0,365243.0,-784.000000,-694.000000,-724.0,-714.000000
4,100006,22257.840,319979.25,303232.50,66987.0,426639.0,15.0,0.217830,NaN,NaN,-213.5,2005.5,20.0,365243.0,121603.666667,122183.666667,243445.0,243447.666667


In [17]:
num_group_df.shape

(305828, 18)

In [18]:
categorical = pd.get_dummies(prev_app[cat_features])
categorical['SK_ID_CURR'] = prev_app['SK_ID_CURR']
categorical.head()

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,...,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,SK_ID_CURR
0,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,271877
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,108129
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,122040
3,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,176158
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,202054


In [19]:
cat_group_df = categorical.groupby('SK_ID_CURR').agg('sum').reset_index()
cat_group_df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,...,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,100001,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,100002,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,100003,1,1,0,0,1,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,100004,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,100006,1,1,2,0,0,0,0,1,3,...,1,0,1,0,0,0,0,0,0,0


In [20]:
cat_group_df.shape

(305828, 142)

In [21]:
flag = prev_app[flag_col]

In [22]:
flag['SK_ID_CURR'] = prev_app['SK_ID_CURR']

C:\Users\dbda\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
flag = pd.get_dummies(flag)

In [24]:
flag.head()

,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL,SK_ID_CURR,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y
0,1,0.0,271877,0,1
1,1,1.0,108129,0,1
2,1,1.0,122040,0,1
3,1,1.0,176158,0,1
4,1,NaN,202054,0,1


In [25]:
flag_group_df = flag.groupby('SK_ID_CURR').agg('sum').reset_index()

In [26]:
flag_group_df[flag_group_df['FLAG_LAST_APPL_PER_CONTRACT_N']==1]

,SK_ID_CURR,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y
46,100059,9,0.0,1,9
72,100091,1,0.0,1,1
228,100270,6,0.0,1,6
325,100382,4,2.0,1,4
374,100437,7,2.0,1,7
414,100485,4,0.0,1,3
583,100681,6,0.0,1,6
593,100695,6,1.0,1,6
618,100722,2,0.0,1,2
622,100727,3,3.0,1,3


In [27]:
flag_group_df.shape

(305828, 5)

In [ ]:
prev_app = num_group_df.merge(cat_group_df,on='')